# 若干种类的支持向量机之练习(Multiclass Support Vector Machine exercise)

本次练习你将会：

- 实现“全向量式(fully-vectorized)”的 SVM **损失函数（loss function）**
- 实现 SVM 的** 分析梯度（analytic gradient）**“全向量式”表达式
- 使用数值梯度（numerical gradient）来检验实现之正确与否
- 使用验证集来**调整学习率和正则化强度（译者注：和前面的任务类似）**
- 通过**SGD**来**优化**损失函数
- **可视化**最终的学习权重

In [3]:
import random 
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] ='nearest'
plt.rcParams['image.cmap'] ='gray'

%load_ext autoreload
%autoreload 2


## CIFAR-10 数据加载和预处理

In [4]:
# 加载原始的CIFAR-10数据
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
# 检查训练集和测试集的大小
print("训练集输入的大小："+str(X_train.shape))
print("训练集标签的大小："+str(y_train.shape))
print("测试集输入的大小："+str(X_test.shape))
print("测试集标签的大小："+str(y_test.shape))

训练集输入的大小：(50000L, 32L, 32L, 3L)
训练集标签的大小：(50000L,)
测试集输入的大小：(10000L, 32L, 32L, 3L)
测试集标签的大小：(10000L,)


In [ ]:
# 将数据集中每个类中抽取几幅来显示
classes = ['plane', 'car', 'bird']